In [77]:
import nltk
from nltk.corpus import brown
import numpy as np

In [110]:
# Let us add the project package into the model, so we can share 
# code with iPython.
import sys 
project_package = '/home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/'
if project_package not in sys.path:
    sys.path.append(project_package)
from shared import utils
from grasshopper import model as GH

In [125]:
reload(GH)
reload(utils)

<module 'shared.utils' from '/home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/shared/utils.pyc'>

In [126]:
original = brown.sents('ca19')
cleanDoc, mapping = utils.cleanDocument(original)
W = GH.docToMatrix(original, sim_fun=utils.cosineSim)
WClean = GH.docToMatrix(cleanDoc, sim_fun=utils.cosineSim)

In [127]:
r = np.ones(len(W))
r = r / np.sum(r)
lsum = 5
lamb = 1.0
results = GH.grasshopper(W, r, lamb, lsum)

In [128]:
r = np.ones(len(WClean)) / len(WClean)
resultsClean = GH.grasshopper(WClean, r, lamb, lsum)

In [129]:
print results
print resultsClean

[(1, 0.0088495575221239735), (70, 0.77597982933701826), (102, 0.77079185117634308), (2, 0.7595610384224718), (18, 0.76076853879791617)]
[(89, 0.008928571428571612), (0, 0.68978929947363266), (92, 0.64265004572368112), (37, 0.64358354257179529), (6, 0.63050721303451929)]


In [130]:
for (i, p) in sorted(resultsClean, key=lambda x: x[0]):
    print ' '.join(brown.sents('ca19')[i])

The Baltimore and Ohio Railroad announced yesterday it would reduce the total amount of its payroll by 10 per cent through salary cuts and lay-offs effective at 12:01 A.M. next Saturday .
A flat reduction of 10 per cent in the salary of all officers , supervisors and other employees not belonging to unions .
For a number of years the board used a machine to keep a permanent record but abandoned the practice about two years ago .
Dodge reported the following breakdown :
and heavy engineering at $9,841,000 , down 45 per cent .


In [131]:
for (i, p) in sorted(results, key=lambda x: x[0]):
    print ' '.join(brown.sents('ca19')[i])

The current monthly payroll comes to about $15,000,000 .
Howard E. Simpson , the railroad's president , said , `` A drastic decline in freight loading due principally to the severe slump in the movement of heavy goods has necessitated this regrettable action '' .
In another attack , Samuel Verstandig , 41 , proprietor of a food store in the 2100 block Aiken Street , told police two Negroes assaulted him in his store and stole $150 from the cash register after choking and beating him .
Soon after 10 A.M. , when police reached the 1-1/2-story brick home in the Franklin Manor section , 15 miles south of here on the bay , in response to a call from the Dresbach's other son , Lee , 14 , they found Mrs. Dresbach's body on the first-floor bedroom floor .
Although economic and personal circumstances vary widely among those now choosing apartments , Leo J. Pantas , vice president of a hardware manufacturing company , pointed out recently that many apartment seekers seem to have one characterist